In [19]:
import pandas as pd
from scipy.stats import binomtest, ttest_ind, f_oneway

Информация о датасете:

FIPS (Federal Information Processing Standards) — это система кодирования, используемая в США для уникальной идентификации географических объектов.

State Postal обычно относится к почтовым аббревиатурам для штатов в США, которые используются в почтовых адресах и официальных документах. 

In [17]:
df = pd.read_csv('2012_US_elect_county.csv', sep=',')
df.drop_duplicates()
df

,State Postal,County Name,FIPS,Obama vote,%,Romney vote,%.1
0,AK,Alaska,0,"91,696",41.6,"121,234",55
1,AK,Alaska,2000,"91,696",41.6,"121,234",55
2,AL,Alabama,0,"793,620",38.4,"1,252,453",60.7
3,AL,Autauga,1001,"6,354",26.6,"17,366",72.6
4,AL,Baldwin,1003,"18,329",21.6,"65,772",77.4
...,...,...,...,...,...,...,...
4069,WV,Webster,54101,939,34.2,"1,708",62.2
4070,WV,Wetzel,54103,"2,194",37.8,"3,449",59.4
4071,WV,Wirt,54105,673,31.1,"1,422",65.7
4072,WV,Wood,54107,"11,067",32.9,"21,980",65.4


# Биномиальный тест

Нулевая гипотеза: Вероятность встретить высокий процент у Obama vote выше, чем встретить низкий процент(то есть за Obama чаще голосуют)

Альтернативная гипотеза: Вероятность встретить низкий процент у Obama vote выше, чем встретить высокий процент(то есть за Romney чаще голосуют)

In [18]:
df['%'] = pd.to_numeric(df['%'], errors='coerce')  # Преобразуем строки в числа, игнорируя ошибки

# Определяем порог для высокого и низкого процента голосов за Обаму
threshold = 50 

# Подсчитываем количество округов с высоким и низким процентом голосов за Обаму
high_obama_votes = (df['%'] > threshold).sum() 
low_obama_votes = (df['%'] <= threshold).sum()

# Биномиальный тест
p_value = binomtest(high_obama_votes, n=high_obama_votes + low_obama_votes, alternative='less')

# Выводим результаты
print(f"Количество округов с высоким процентом голосов за Обаму: {high_obama_votes}")
print(f"Количество округов с низким процентом голосов за Обаму: {low_obama_votes}")
print(f"P-value биномиального теста: {p_value}")

# Интерпретация результата
if p_value < 0.05:
    print("Отвергаем нулевую гипотезу. Вероятность встретить низкий процент голосов за Обаму выше.")
else:
    print("Не можем отвергнуть нулевую гипотезу. Вероятность встретить высокий процент голосов за Обаму выше.")


Количество округов с высоким процентом голосов за Обаму: 1434
Количество округов с низким процентом голосов за Обаму: 2638
P-value биномиального теста: 9.66895251504628e-81
Отвергаем нулевую гипотезу. Вероятность встретить низкий процент голосов за Обаму выше.


C:\Users\Redmi\AppData\Local\Temp\ipykernel_23180\2017131209.py:14: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(high_obama_votes, n=high_obama_votes + low_obama_votes, alternative='less')


# T-test

Переменная для разделения на группы: деление по результатам выборов за Обаму и Ромни.

Цель сравнения: определить, есть ли значимая разница между средними процентами голосов за Обаму и Ромни в выборке.

Нулевая гипотеза: средние проценты голосов за Обаму и Ромни в данном округе не отличаются.

In [20]:
# Преобразуем проценты голосов за Обаму и Ромни в числовой формат
df['Obama %'] = pd.to_numeric(df['%'], errors='coerce')  # Процент за Обаму
df['Romney %'] = pd.to_numeric(df['%.1'], errors='coerce')  # Процент за Ромни

# Удаляем строки с пропущенными значениями в этих столбцах
df_clean = df.dropna(subset=['Obama %', 'Romney %'])

# Выполняем T-тест
t_stat, p_value = ttest_ind(df_clean['Obama %'], df_clean['Romney %'], alternative='two-sided')

# Выводим результаты
print(f"T-статистика: {t_stat}")
print(f"P-value: {p_value}")

# Интерпретация результата
if p_value < 0.05:
    print("Отвергаем нулевую гипотезу. Средние проценты голосов за Обаму и Ромни значимо отличаются.")
else:
    print("Не можем отвергнуть нулевую гипотезу. Средние проценты голосов за Обаму и Ромни не отличаются значимо.")


T-статистика: -28.317717172174444
P-value: 2.461715259009671e-168
Отвергаем нулевую гипотезу. Средние проценты голосов за Обаму и Ромни значимо отличаются.


# ANOVA

Мы хотим выяснить, есть ли значимые различия в поддержке Обамы (в процентах) между различными штатами.

Нулевая гипотеза: Нет значимых различий в проценте голосов за Обаму между штатами.

Альтернативная гипотеза: Есть значимые различия в проценте голосов за Обаму между штатами.

In [22]:
# Преобразуем проценты голосов за Обаму в числовой формат
df['Obama %'] = pd.to_numeric(df['%'], errors='coerce')

# Удаляем строки с пропущенными значениями в этих столбцах
df_clean = df.dropna(subset=['Obama %', 'State Postal'])

# Группируем данные по штату
grouped_data = df_clean.groupby('State Postal')['Obama %'].apply(list)

# Выполняем тест ANOVA
f_stat, p_value = f_oneway(*grouped_data)

# Выводим результаты
print(f"F-статистика: {f_stat}")
print(f"P-value: {p_value}")

# Интерпретация результата
if p_value < 0.05:
    print("Отвергаем нулевую гипотезу. Так как значения сильно отличаются друг от друга в проценте голосов за Обаму между штатами.")
else:
    print("Не можем отвергнуть нулевую гипотезу. Различий в проценте голосов за Обаму между штатами нет.")


F-статистика: 92.16331347439103
P-value: 0.0
Отвергаем нулевую гипотезу. Есть значимые различия в проценте голосов за Обаму между штатами.


### Выводы по результатам тестов:

1. **Биномиальный тест**:
   - **Описание метода**: Биномиальный тест используется для проверки гипотезы о том, что вероятность одного из двух исходов (в данном случае высокого или низкого процента голосов за Обаму) превышает другую.
   - **Результаты**: Количество округов с высоким процентом голосов за Обаму — 1434, с низким процентом — 2638. P-value биномиального теста составило \(9.67 \times 10^{-81}\), что значительно меньше уровня значимости 0.05.
   - **Вывод**: Нулевая гипотеза о том, что вероятность высокого процента голосов за Обаму выше, отвергается. Тест показал, что в большем количестве округов процент голосов за Обаму ниже 50%, что говорит о преобладании низкой поддержки.

2. **T-тест (тест Стьюдента)**:
   - **Описание метода**: T-тест используется для сравнения средних значений двух выборок (в данном случае процентов голосов за Обаму и за Ромни). Он проверяет, есть ли значимые различия между этими средними.
   - **Результаты**: T-статистика: -28.32, P-value: \(2.46 \times 10^{-168}\), что значительно меньше 0.05.
   - **Вывод**: Нулевая гипотеза о равенстве средних значений процентов голосов за Обаму и Ромни отвергается. T-тест выявил значительные различия в процентах голосов, что означает, что поддержка обоих кандидатов распределилась неравномерно.

3. **ANOVA (дисперсионный анализ)**:
   - **Описание метода**: ANOVA используется для проверки наличия значимых различий между несколькими группами (в данном случае процент голосов за Обаму в разных штатах). Тест проверяет, отличаются ли средние значения между группами больше, чем можно ожидать случайно.
   - **Результаты**: F-статистика: 92.16, P-value: 0.0.
   - **Вывод**: Нулевая гипотеза о том, что процент голосов за Обаму одинаков между штатами, отвергается. ANOVA показал, что между штатами существуют значимые различия в поддержке Обамы, то есть проценты голосов за Обаму варьируются в зависимости от штата.

### Общий акцент на методы:
Каждый тест имеет свою область применения, и в данной работе они применены последовательно для проверки различных гипотез:
- **Биномиальный тест** подтвердил различие в вероятностях высокого и низкого процента голосов.
- **T-тест** выявил существенное различие в средних процентах голосов за Обаму и Ромни.
- **ANOVA** показал наличие значимых различий в уровне поддержки Обамы между штатами.

Эти тесты продемонстрировали свою эффективность для анализа процентных данных и выявления закономерностей в распределении голосов.